# Preprocessing intracranial EEG using MNE-python


*WIRED 2024*  
[Maansi Desai, PhD](https://maansidesai.github.io/)  
Postdoctoral Researcher in the [Hamilton Lab](https://slhs.utexas.edu/research/hamilton-lab)
<br>
Department of Speech, Language, and Hearing Sciences 
<br>
The University of Texas at Austin 

## Part 3: Plotting electrodes on brain using MNE brain.viz function


In [ ]:


import mne
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
from mne_bids import read_raw_bids, print_dir_tree, BIDSPath
from mne_bids.path import get_bids_path_from_fname
from bids import BIDSLayout
from ecog_preproc_utils import transformData
import bids 
import re  # regex for comparing channel names

## Download BIDS iEEG dataset

Here we will download an example iEEG dataset from [this dataset from the Hamilton lab](https://openneuro.org/datasets/ds004993/versions/1.1.1). If you have ran [`01_ieeg_preprocessing_MNE`](01_ieeg_preprocessing_MNE) and [`02_ieeg_preprocessing_MNE_epochs.ipynb`](02_ieeg_preprocessing_MNE_epochs.ipynb), then you should already have downloaded the necessary datasets locally. 

For this notebook we will use data from: 
 - `sub-W1`, `iemu`, `B3`, `movietrailers`
 - `sub-W2`, `iemu`, `B8`, `timit5`
 - `sub-W3`, `iemu`, `B8`, `timit4`

In [ ]:
# This is the example participant's data that we will load for the tutorial,
# but there are more options.
subj = 'W2'
sess = 'iemu'
task = 'timit5'
acq = 'B8'
run = '01'

In [ ]:
import mne_bids
mne_bids.__version__

In [ ]:
# Change the data directory below to where your data are located. 
parent_dir = 'data/ds004993'  # This is on your local machine

bids_path = BIDSPath(
    root=parent_dir, subject=subj, session=sess, task=task, acquisition=acq, run=run, datatype="ieeg"
)
print(bids_path)

## BIDS layout

We can use `pybids` to show a little bit about the files in this BIDS dataset. We won't get as much into this, but if you'd like to try this tutorial on your own you may wish to delve into this more.

In [ ]:
layout = BIDSLayout(parent_dir)

In [ ]:
layout.get_tasks()

In [ ]:
all_files = layout.get()
print("There are {} files in the layout.".format(len(all_files)))
print("\nThe first 10 files are:")
all_files[:10]

In [ ]:
print_dir_tree(parent_dir, max_depth=3)

## Let's load some iEEG data!

In [ ]:
# Read data and extract parameters from BIDS files
raw = read_raw_bids(bids_path, verbose=True)

## Plot the raw data

In [ ]:
raw.plot(scalings='auto', color=dict(eeg='b', ecog='b'), n_channels=64, block=True)

## Plot the power spectrum

In [ ]:
raw.compute_psd().plot(picks='data', exclude=[]);

# Electrodes on the brain


In [ ]:
%matplotlib qt
brain_kwargs = dict(alpha=0.1, background="white", view='split',cortex="low_contrast")
brain = mne.viz.Brain(f'sub-{subj}', surf='pial', subjects_dir=f'{parent_dir}/freesurfer', alpha=0.5,
                      background='white')
trans = mne.read_trans(f'{parent_dir}/freesurfer/sub-{subj}/bem/sub-{subj}_head-trans.fif')
brain.add_sensors(raw.info, trans=trans)

## Get the high gamma data

In [ ]:
# Get only the iEEG channels for high gamma
raw_ieeg = raw.copy()
raw_ieeg.pick_types(seeg=True)
raw_ieeg.anonymize()

notch_freqs = list(np.arange(raw.info['line_freq'], raw.info['lowpass'], step=raw.info['line_freq']))
# Get the high gamma data
# Generally, do a CAR if you have widespread coverage over multiple
# areas (not just one sensory area)
# If you have limited coverage, you may choose to do no CAR or choose
# to reference to one specific channel.
ieeg_dir = f'{parent_dir}/sub-{subj}/ses-{sess}/ieeg/'
hgdat = transformData(raw_ieeg, ieeg_dir, band='high_gamma', notch=True, CAR=True,
                      car_chans='average', log_transform=True, do_zscore=True,
                      hg_fs=100, notch_freqs=notch_freqs, ch_types='seeg')

## Get the events from BIDS file

In [ ]:
events = mne.events_from_annotations(hgdat, event_id='auto')

In [ ]:
#let's plot the neural response for one electrode
epochs = mne.Epochs(hgdat, events[0], tmin=-0.5, tmax=0.6)
epochs.plot_image(picks=[hgdat.info['ch_names'][49]])

## We can use the function `epochs.get_data()` to get the values from our evoked data and normalize these values across channels. Then, create a dictionary of colors which represent which electrodes have the strongest responses to the onset of the speech sentences. We'll use this dictionary to plot these responses on the brain

In [ ]:
edat=epochs.get_data().mean(0).mean(1)
print(edat.shape)
norm_edat = edat/edat.max()
norm_edat[norm_edat<0]=0
col = np.zeros((len(raw.info['ch_names']),3))
col[:,0]=norm_edat
colors = dict(seeg=col)

## Plot electrodes just colored by their average high gamma during sentence listening.

The more red, the more speech-responsive the electrodes are in the brain

In [ ]:
brain_kwargs = dict(alpha=0.1, background="white", view='split',cortex="low_contrast")
brain = mne.viz.Brain(f'sub-{subj}', surf='pial', subjects_dir=f'{parent_dir}/freesurfer', alpha=0.5,\
                      background='white')
trans = mne.read_trans(f'{parent_dir}/freesurfer/sub-{subj}/bem/sub-{subj}_head-trans.fif')
brain.add_sensors(raw.info, trans=trans, sensor_colors=colors)